In [0]:
from graphframes import GraphFrame
from pyspark.sql.functions import col, desc
from pyspark.sql import functions as F
import pandas as pd

In [0]:
#data = spark.read.csv('/FileStore/tables/soc_sign_bitcoinalpha.csv', inferSchema=True, header=True)
url = "https://raw.githubusercontent.com/preetika19/Analyzing-Social-Networks-using-GraphFrame/main/soc-sign-bitcoinalpha.csv"
data = spark.createDataFrame(pd.read_csv(url))
data.count()

Out[2]: 24186

In [0]:
vertices = data.dropna().selectExpr('explode(array(Source, Target))').distinct()
vertices = vertices.select(col('col').alias('id'))
vertices.show(5)

+----+
|  id|
+----+
| 430|
|   1|
|3010|
|3134|
|7188|
+----+
only showing top 5 rows



In [0]:
edges = data.select(["Source", "Target"]).withColumnRenamed("Source", "src").withColumnRenamed("Target", "dst")
edges.show(5)

+----+---+
| src|dst|
+----+---+
|7188|  1|
| 430|  1|
|3134|  1|
|3026|  1|
|3010|  1|
+----+---+
only showing top 5 rows



In [0]:
# a. Find the top 5 nodes with the highest outdegree and find the count of the number of outgoing edges in each
graph = GraphFrame(v = vertices, e = edges)
graph.outDegrees.sort(desc('outDegree')).show(5)

/databricks/spark/python/pyspark/sql/dataframe.py:150: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
/databricks/spark/python/pyspark/sql/dataframe.py:129: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")
+---+---------+
| id|outDegree|
+---+---------+
|  1|      490|
|  8|      259|
|  3|      243|
|  4|      215|
|  7|      212|
+---+---------+
only showing top 5 rows



In [0]:
# b. Find the top 5 nodes with the highest indegree and find the count of the number of incoming edges in each
graph.inDegrees.sort(desc('inDegree')).show(5)

+---+--------+
| id|inDegree|
+---+--------+
|  1|     398|
|  3|     251|
|  2|     205|
| 11|     203|
|  4|     201|
+---+--------+
only showing top 5 rows



In [0]:
# c. Calculate PageRank for each of the nodes and output the top 5 nodes with the highest PageRank values.
pageRank = graph.pageRank(maxIter=20)
pageRank.vertices.orderBy(desc("pagerank")).select("id", "pagerank").show(5)    

+---+------------------+
| id|          pagerank|
+---+------------------+
|  1| 64.24495154666965|
|  3| 33.96160879196087|
|  4|30.400464805730472|
|  2|25.106833340153823|
|177| 25.05932373630836|
+---+------------------+
only showing top 5 rows



In [0]:
# d. Run the connected components algorithm on it and find the top 5 components with the largest number of nodes.
spark.sparkContext.setCheckpointDir('/FileStore/checkpoints')
components = graph.connectedComponents()
display(components.groupBy('component').agg(F.collect_list('id').alias('subnetworks'),F.count('id').alias('total_count')).orderBy(desc('total_count')).limit(5))

component subnetworks total_count 1 List(964, 474, 29, 2927, 26, 1840, 2173, 2941, 65, 1360, 1010, 191, 3009, 7544, 222, 2909, 2989, 293, 1409, 1127, 270, 7583, 1642, 1551, 1371, 3155, 1936, 1175, 7595, 720, 367, 278, 2371, 296, 54, 19, 1782, 1336, 348, 277, 287, 2881, 2244, 1921, 656, 1055, 112, 113, 736, 155, 299, 347, 241, 237, 564, 1547, 1636, 602, 3158, 2378, 2966, 330, 22, 2488, 1338, 3200, 2263, 198, 2282, 196, 1059, 2439, 1374, 1157, 1152, 1353, 1760, 1505, 2775, 7, 77, 532, 34, 3341, 1542, 2181, 7566, 1445, 2942, 1458, 1483, 2700, 188, 2300, 3092, 126, 529, 2497, 228, 274, 202, 1768, 2227, 7425, 262, 3330, 734, 691, 2308, 1415, 570, 1908, 1776, 3339, 2772, 2764, 1174, 1816, 1038, 149, 2149, 1302, 264, 2119, 1632, 1064, 876, 638, 1606, 1846, 3173, 650, 1376, 1185, 439, 190, 110, 706, 2987, 57, 136, 3105, 3043, 412, 3133, 1571, 417, 2192, 3210, 2235, 1025, 2784, 1283, 2404, 2336, 1051, 1128, 43, 420, 32, 752, 1550, 1108, 696, 84, 1233, 2276, 2926, 1362, 2358, 958, 31, 1361, 395, 119, 768, 7400, 974, 2113, 596, 326, 386, 2709, 1461, 1069, 2923, 2806, 561, 39, 7577, 7401, 7528, 98, 7598, 2185, 402, 1548, 2360, 7516, 1282, 773, 2397, 2996, 1482, 2499, 2839, 203, 2922, 3319, 2472, 116, 1942, 145, 2285, 1602, 2742, 1826, 25, 3275, 1757, 3156, 2999, 1731, 2261, 1347, 586, 2334, 124, 2687, 3295, 1368, 181, 1535, 7563, 2384, 1785, 1737, 686, 95, 2176, 562, 268, 589, 227, 699, 455, 915, 1395, 71, 370, 2763, 2346, 621, 703, 1274, 2903, 642, 161, 365, 193, 2354, 2869, 2586, 68, 1520, 2498, 6, 466, 2740, 87, 2177, 72, 7552, 3328, 215, 58, 255, 258, 2795, 1098, 3198, 320, 883, 1862, 271, 107, 424, 2291, 266, 882, 221, 554, 9, 744, 1570, 158, 2281, 2920, 796, 389, 1693, 63, 754, 27, 1516, 1267, 2743, 51, 56, 7126, 3042, 178, 353, 702, 891, 3193, 1923, 3418, 341, 3253, 150, 1545, 146, 346, 52, 1849, 459, 2898, 585, 446, 7509, 3378, 1046, 174, 712, 825, 946, 17, 1060, 1342, 2826, 2786, 79, 170, 209, 661, 1346, 2338, 3004, 236, 103, 7327, 253, 7597, 276, 509, 7510, 1287, 139, 7591, 430, 3355, 817, 2695, 242, 1009, 1230, 7536, 3139, 1574, 7482, 1085, 41, 519, 2848, 3163, 257, 208, 88, 33, 2201, 248, 384, 2209, 637, 1358, 7341, 207, 1576, 3309, 138, 3011, 649, 3300, 462, 304, 769, 2853, 3148, 114, 646, 434, 599, 2376, 7517, 3329, 2383, 3247, 1331, 5, 1363, 1, 1583, 1316, 896, 2876, 1875, 1909, 319, 115, 2619, 2293, 718, 153, 7585, 377, 96, 250, 3274, 3143, 1136, 3259, 1367, 859, 2492, 1218, 7550, 743, 1877, 3103, 2747, 890, 1560, 3132, 1790, 3189, 3113, 185, 375, 1054, 632, 349, 7503, 623, 2820, 1436, 944, 1232, 2296, 219, 7500, 104, 1151, 7590, 1493, 1024, 214, 1284, 449, 7406, 1536, 538, 874, 2779, 1475, 2707, 7508, 818, 7377, 419, 1373, 10, 120, 748, 423, 1087, 3019, 2210, 89, 2178, 3349, 2830, 7380, 521, 2147, 3049, 361, 2238, 2305, 7348, 1487, 501, 2868, 3236, 1566, 452, 1066, 1822, 871, 356, 1867, 697, 3013, 197, 1239, 1322, 497, 2808, 2367, 3283, 3084, 85, 106, 111, 815, 1456, 1310, 3233, 3118, 301, 2810, 7525, 411, 177, 6369, 3070, 2090, 3421, 3048, 2006, 1558, 460, 1832, 2298, 2511, 563, 179, 1251, 2163, 306, 1222, 557, 2017, 7551, 2315, 67, 517, 2353, 7394, 755, 224, 1295, 142, 2559, 168, 5679, 749, 2405, 7514, 3145, 247, 100, 528, 318, 105, 131, 3365, 3290, 654, 7479, 333, 3211, 1736, 108, 804, 309, 44, 1793, 3006, 471, 1068, 123, 951, 683, 641, 345, 1815, 3229, 1117, 590, 1555, 7522, 7442, 3141, 994, 1998, 205, 7523, 1885, 2632, 1678, 323, 1308, 249, 1856, 1842, 3256, 381, 3167, 1812, 1309, 3127, 61, 1392, 379, 777, 988, 1348, 2335, 3057, 567, 535, 7602, 454, 473, 1264, 1192, 156, 1691, 1524, 1107, 841, 1153, 1340, 636, 587, 846, 429, 160, 3392, 3332, 2198, 1977, 7578, 1254, 3112, 1900, 2605, 3, 435, 2024, 982, 979, 206, 382, 7594, 3010, 37, 1465, 292, 3083, 1090, 1372, 431, 1580, 2870, 2351, 3059, 83, 2333, 362, 491, 2102, 2858, 355, 200, 295, 12, 2306, 2352, 2952, 2746, 543, 879, 288, 826, 141, 1538, 373, 544, 1056, 2328, 614, 2142, 74, 492, 7604, 2002, 664, 893, 280, 1378, 7444, 3134, 1261, 62, 8, 658, 182, 512, 1853, 2302,

In [0]:
# e. Run the triangle counts algorithm on each of the vertices and output the top 5 vertices with the largest triangle count. In case of ties, you can randomly select the top 5 vertices.
triangle_count = graph.triangleCount()
triangle_count.select("id", "count").orderBy(desc('count')).show(5)

+---+-----+
| id|count|
+---+-----+
| 11| 1815|
|  2| 1628|
|177| 1414|
|  3| 1336|
|  7| 1181|
+---+-----+
only showing top 5 rows

